## Read Volvo CSV file

In the Volvo "On Call" App in the "Driving Journal" section, your last 100 days of
trips are recorded.  This includes miles and time travelled, destinations,
fuel consumption, etc.

The Volvo "On Call" App gives you the option to download your data in a text,
csv, or Excel file.

This program takes that information and calculates summary statistics.

I used the csv option.  Here are a few notes:

* To make the downloaded csv file readable by pandas (a python library used to crunch
  lots of data), import the file into Google Sheets (set the delimiter
  to a ";".  Then export the file as a csv file.  This csv will be readable by pandas.
* If you don't have access to Google Sheets, read the file using the
  code below, copy/paste the output to another file called Volvo.csv. Pandas can read
  the new file.
* There are 4 columns that don't contain data, so I dropped them.
* The Started and Stopped columns were converted to actual dates and the time Duration
  was converted to minutes instead of the colon notation (hours:minutes).

In [85]:
import pandas as pd

In [86]:
# The Volvo csv file needs cleanup.
# Run this, print the lines, and copy/paste the output to a file
# called Volvo.csv.
#
with open("Volvo-jan-10-2021.csv", encoding="utf8", errors='ignore') as f:
    lines = f.read().splitlines()
    for i in lines:
        # Remove the "pass" line and uncomment the print line
        # so you can see lines.  Then copy/paste the output
        # to a text file called Volvo.csv.
        #
        pass
        #print(i)

In [87]:
# Read the resulting .csv file using pandas.
#
df = pd.read_csv("Volvo.csv", delimiter=';')

In [88]:
# Show list of columns so we know what to drop (in my case,
# several columns are empty).
#
df.columns

Index(['Category', 'Started', 'Start odometer (miles)', 'Start address',
       'Stopped', 'End odometer (miles)', 'End address', 'Duration',
       'Distance (miles)', 'Fuel consumption (gallons)', 'Title', 'User Notes',
       'Unnamed: 12'],
      dtype='object')

In [89]:
# Drop unused/empty columns.
#
for col in ['Category', 'Title', 'User Notes', 'Unnamed: 12']:
    df.drop(col, axis=1, inplace=True)

In [90]:
# Transform Started/Stop to actual dates
#
df.Started = pd.to_datetime(df.Started)
df.Stopped = pd.to_datetime(df.Stopped)

# Transform Duration (in the form of "xx:xx h" into integer minutes).
#
df['Duration'] = df['Duration'].apply(lambda x: int(x[:-2].split(':')[0]) * 60 + int(x[:-2].split(":")[1]))

In [174]:
# Show the data (omitting addresses for the data donor's privacy).
#
df[['Started', 'Start odometer (miles)',
       'Stopped', 'End odometer (miles)', 'Duration',
       'Distance (miles)', 'Fuel consumption (gallons)']].head()

,Started,Start odometer (miles),Stopped,End odometer (miles),Duration,Distance (miles),Fuel consumption (gallons)
0,2021-01-09 16:53:00,4019,2021-01-09 17:08:00,4025,14,6.559,0.258
1,2021-01-09 16:26:00,4012,2021-01-09 16:39:00,4019,13,6.554,0.264
2,2021-01-09 11:26:00,4003,2021-01-09 11:45:00,4012,19,9.573,0.414
3,2021-01-09 10:53:00,3985,2021-01-09 11:23:00,4003,30,17.790,0.657
4,2021-01-09 10:41:00,3983,2021-01-09 10:47:00,3985,6,1.825,0.097


In [167]:
# CSV data is downloaded for the last 100 days.
# Perform some basic calculations.
#
first_odo = df.iloc[-1, 4]
last_odo = df.iloc[0, 4]
total_miles = last_odo - first_odo

total_time = df['Duration'].sum()
hours = total_time // 60
minutes = total_time % 60
total_fuel = df['Fuel consumption (gallons)'].sum()

ave_trip = df['Distance (miles)'].mean()
ave_duration = df['Duration'].mean()
ave_speed = ave_trip / (ave_duration / 60.0)

In [169]:
print("In the last 100 days:\n")
print(f"  Miles driven   : {total_miles:,d}")
print(f"  Fuel consumed  : {total_fuel:,.2f} gallons")
print(f"  Average Trip   : {ave_trip:,.2f} miles" )
print(f"  Average Time   : {ave_duration:,.2f} minutes")
print(f"  Average Speed  : {ave_speed:,.2f} mph")

print(f"  Overal mpg     : {total_miles / total_fuel:,.2f}")
print(f"  Time driven    : {hours} hours {minutes} minutes")
print()
print(f"  Unique starting points : {len(df['Start address'].unique())}")
print(f"  Unique ending points   : {len(df['End address'].unique())}")

In the last 100 days:

  Miles driven   : 2,422
  Fuel consumed  : 93.02 gallons
  Average Trip   : 9.52 miles
  Average Time   : 17.78 minutes
  Average Speed  : 32.14 mph
  Overal mpg     : 26.04
  Time driven    : 75 hours 34 minutes

  Unique starting points : 75
  Unique ending points   : 70
